In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tqdm
import time
import requests
from io import StringIO
from datetime import datetime
import mplfinance as mpf
import plotly.graph_objects as go
from alpha_vantage.timeseries import TimeSeries

**Get data functions**

In [74]:
def get_daily_data(ticker, is_saved):
    
    time_length = 'TIME_SERIES_DAILY'
    datatype = 'csv'
    
    url = f'https://www.alphavantage.co/query?function={time_length}&symbol={ticker}&outputsize=full&apikey=LSY1P7SPJOP4UKMB&datatype={datatype}'
    
    response = requests.get(url)

    data_string = StringIO(response.text)
    data = pd.read_csv(data_string)[::-1]
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data.rename(columns={'timestamp': 'date'}, inplace=True)
    data.set_index('date', drop=True, inplace=True)
    
    return data

def get_hourly_data(ticker, start_year, end_year, is_saved):
    
    time_length = 'TIME_SERIES_INTRADAY'
    interval_length = '60min'
    datatype = 'csv'
    
    hourly_data = pd.DataFrame()
    
    for year in range(start_year, end_year+1):
        for month in range(1,13):
            if month < 10:
                timestamp = f'{year}-0{month}'
            else:
                timestamp = f'{year}-{month}'
            
            url = f'https://www.alphavantage.co/query?function={time_length}&symbol={ticker}&interval={interval_length}&outputsize=full&apikey=LSY1P7SPJOP4UKMB&datatype={datatype}&month={timestamp}'
            response = requests.get(url)

            data_string = StringIO(response.text)
            df = pd.read_csv(data_string)[::-1]
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df.rename(columns={'timestamp': 'date'}, inplace=True)
            df.set_index('date', drop=True, inplace=True)
            hourly_data = pd.concat([hourly_data, df], axis=0)
            time.sleep(13)
    
    return hourly_data

def get_5min_data(ticker, start_year, end_year, is_saved):
    
    time_length = 'TIME_SERIES_INTRADAY'
    interval_length = '5min'
    datatype = 'csv'
    
    min_data = pd.DataFrame()
    
    for year in range(start_year, end_year+1):
        for month in range(1,13):
            if month < 10:
                timestamp = f'{year}-0{month}'
            else:
                timestamp = f'{year}-{month}'
            
            url = f'https://www.alphavantage.co/query?function={time_length}&symbol={ticker}&interval={interval_length}&outputsize=full&apikey=LSY1P7SPJOP4UKMB&datatype={datatype}&month={timestamp}'
            response = requests.get(url)

            data_string = StringIO(response.text)
            df = pd.read_csv(data_string)[::-1]
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df.rename(columns={'timestamp': 'date'}, inplace=True)
            df.set_index('date', drop=True, inplace=True)
            min_data = pd.concat([min_data, df], axis=0)
            time.sleep(13)
    
    return min_data

In [75]:
data = get_5min_data('AAPL', 2020,2020,False)

In [76]:
data

,open,high,low,close,volume
date,,,,,
2020-01-02 04:00:00,72.055,72.075,72.031,72.060,31832
2020-01-02 04:05:00,72.128,72.239,72.104,72.223,6536
2020-01-02 04:10:00,72.189,72.224,72.165,72.209,5012
2020-01-02 04:15:00,72.209,72.237,72.185,72.221,1752
2020-01-02 04:20:00,72.189,72.237,72.165,72.221,8672
...,...,...,...,...,...
2020-12-31 19:35:00,130.778,130.832,130.751,130.814,2552
2020-12-31 19:40:00,130.817,130.862,130.771,130.824,4495
2020-12-31 19:45:00,130.807,130.832,130.731,130.755,9738


**Technical Indicators**

In [77]:
def camarilla_indicator(data):
    data_shifted = data.shift(1)  # Shifting the data one step upward

    data['R4'] = data_shifted['close'] + (data_shifted['high'] - data_shifted['low']) * 1.1/2
    data['R3'] = data_shifted['close'] + (data_shifted['high'] - data_shifted['low']) * 1.1/4
    data['S3'] = data_shifted['close'] - (data_shifted['high'] - data_shifted['low']) * 1.1/4
    data['S4'] = data_shifted['close'] - (data_shifted['high'] - data_shifted['low']) * 1.1/2

In [78]:
def ma_indicator(data, window):
    data_shifted = data.shift(1)
    
    data[f'MA {window}'] = data_shifted['close'].rolling(window).mean()

In [79]:
def camarilla_check(data):
    today_camarilla = data.shift(1)[['R4', 'R3', 'S3', 'S4']]
    today_camarilla.columns = ['PR4', 'PR3', 'PS3', 'PS4']
    concat = pd.concat([data, today_camarilla], axis=1)

    data['narrow'] = 0
    data.loc[(concat['PR4'] > concat['R4']) & (concat['PR3'] > concat['R3']) & (concat['PS4'] < concat['S4']) & (concat['PS3'] < concat['S3']), 'narrow'] = 1

**Database functions**

In [80]:
def convert_to_datetime(date):
    return pd.to_datetime(date)

def get_date(date_and_time):
    return str(date_and_time).split(' ')[0]

def get_hour(date_and_time):
    timestamp = str(date_and_time).split(' ')[1]
    date = get_date(date_and_time)
    complete_hour = date + ' ' + timestamp.split(':')[0] + ':00:00'
    return complete_hour

In [81]:
def add_hourly_to_5min(database, tickers):
    for ticker in tickers:
        five_min_data = database[ticker]['5min']
        hourly_data = database[ticker]['hourly']
        for index in five_min_data.index:
            hour = get_hour(index)
            
            if date in hourly_data.index:
                five_min_data.loc[index, 'daily close'] = hourly_data.loc[hour, 'close']
                five_min_data.loc[index, 'R4'] = hourly_data.loc[hour, 'R4']
                five_min_data.loc[index, 'R3'] = hourly_data.loc[hour, 'R3']
                five_min_data.loc[index, 'S3'] = hourly_data.loc[hour, 'S3']
                five_min_data.loc[index, 'S4'] = hourly_data.loc[hour, 'S4']
                five_min_data.loc[index, 'narrow'] = hourly_data.loc[hour, 'narrow']

def add_daily_to_hourly(database, tickers):
    for ticker in tickers:
        daily_data = database[ticker]['daily']
        hourly_data = database[ticker]['hourly']
        for index in hourly_data.index:
            date = get_date(index)
            
            if date in daily_data.index:
                hourly_data.loc[index, 'daily close'] = daily_data.loc[date, 'close']
                hourly_data.loc[index, 'R4'] = daily_data.loc[date, 'R4']
                hourly_data.loc[index, 'R3'] = daily_data.loc[date, 'R3']
                hourly_data.loc[index, 'S3'] = daily_data.loc[date, 'S3']
                hourly_data.loc[index, 'S4'] = daily_data.loc[date, 'S4']
                hourly_data.loc[index, 'narrow'] = daily_data.loc[date, 'narrow']
            
def drop_all_na(database, tickers):
    for ticker in tickers:
        daily_data = database[ticker]['daily']
        hourly_data = database[ticker]['hourly']
        daily_data.dropna(inplace=True)
        hourly_data.dropna(inplace=True)
        
def get_data(database, tickers, start_year, end_year):
    for ticker in tickers:
        daily_data = get_daily_data(ticker, False)
        hourly_data = get_hourly_data(ticker, start_year, end_year, False)
        five_min_data = get_5min_data(ticker, start_year, end_year, False)
        database[ticker] = {'daily': daily_data, 'hourly': hourly_data, '5min': five_min_data}
        
def add_indicators(database, tickers):
    for ticker in tickers:
        daily_data = database[ticker]['daily']
        hourly_data = database[ticker]['hourly']
        
        camarilla_indicator(daily_data)
        camarilla_check(daily_data)
        ma_indicator(hourly_data, 50)
        ma_indicator(hourly_data, 200)
        

**Plotting functions**

In [82]:
def plot_data(specific_data, start, end):  
    if start in specific_data.index and end in specific_data.index:
        specific_data = specific_data.loc[start: end]
        fig = go.Figure(specific_data=[go.Candlestick(x=specific_data.index,
                    open=specific_data['open'],
                    high=specific_data['high'],
                    low=specific_data['low'],
                    close=specific_data['close'])])

        fig.show()
    else:
         print("Start or end date not found in the data.")
            
def plot_hourly_data(data, start, end):
    data = data['hourly']
    if start in data.index and end in data.index:
        data = data.loc[start: end]
        
        # Create a subplot layout
        fig, ax = plt.subplots(figsize=(20,10))
        
        # Plot the candlestick chart
        mpf.plot(data, type='candle', ax=ax, show_nontrading=True)
        
        # Plot the 'R4' and 'R3' lines
        ax.plot(data.index, data['R4'], color='red', label='R4', linewidth=1, linestyle='dashed')
        ax.plot(data.index, data['R3'], color='orange', label='R4', linewidth=1, linestyle='dashed')
        ax.plot(data.index, data['S3'], color='blue', label='R4', linewidth=1, linestyle='dashed')
        ax.plot(data.index, data['S4'], color='green', label='R4', linewidth=1, linestyle='dashed')
        ax.plot(data.index, data['MA 200'], color='purple', label='R4', linewidth=1)
        ax.plot(data.index, data['MA 50'], color='navy', label='R4', linewidth=1)
        # Set the y-axis label
        ax.set_ylabel('Stock Price')
        
        # Add a legend
        ax.legend()
        
        # Show the plot
        plt.show()
        
    else:
        print("Start or end date not found in the data.")
        

**Initialization**

In [83]:
tickers = ["SPY"]
database = {}
get_data(database, tickers, 2023, 2023)

KeyError: 'timestamp'

In [ ]:
add_indicators(database, tickers)
add_daily_to_hourly(database, tickers)
add_hourly_to_5min(database, tickers)
drop_all_na(database, tickers)

In [180]:
test = database['SPY']['hourly']

In [182]:
test

,open,high,low,close,volume,MA 50,MA 200,daily close,R4,R3,S3,S4,narrow
date,,,,,,,,,,,,,
2023-03-23 16:00:00,391.681,393.387,391.469,392.865,4477676,394.55724,397.437935,391.709653,396.384590,393.519090,387.788090,384.922590,0.0
2023-03-23 17:00:00,392.897,393.188,391.708,393.005,576169,394.47492,397.413780,391.709653,396.384590,393.519090,387.788090,384.922590,0.0
2023-03-23 18:00:00,392.947,393.078,391.877,392.148,119253,394.38408,397.390625,391.709653,396.384590,393.519090,387.788090,384.922590,0.0
2023-03-23 19:00:00,392.100,392.640,392.087,392.447,75983,394.29894,397.364970,391.709653,396.384590,393.519090,387.788090,384.922590,0.0
2023-03-24 04:00:00,393.385,393.387,390.313,391.361,146206,394.22454,397.340315,394.280070,396.626653,394.168153,389.251153,386.792653,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-13 16:00:00,449.530,449.650,445.843,449.170,12168888,444.27078,442.296600,449.560000,447.433445,446.726722,445.313277,444.606555,0.0
2023-07-13 17:00:00,449.140,449.560,449.140,449.240,189048,444.46108,442.296600,449.560000,447.433445,446.726722,445.313277,444.606555,0.0
2023-07-13 18:00:00,449.240,449.560,449.160,449.420,1258283,444.65298,442.296600,449.560000,447.433445,446.726722,445.313277,444.606555,0.0


In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover

In [104]:
test = database['SPY']['hourly']

In [113]:
concat = check_narrower_camarilla(test)

ValueError: Length mismatch: Expected axis has 998 elements, new values have 4 elements

In [ ]:
class CMA(Strategy):
    
    def init(self):
        
    
    def next(self):
        if crossover(self.data['close'], self.data['MA 200']) 
            and crossover(self.data['close'], self.data['MA 50']):
            if 
            self.position.close()
            self.buy()

        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()